# 5.3 Loading a ship

In [51]:
import pandas as pd
from mms185 import simpson
from mms185.sympy import lambdify,run
import matplotlib.pyplot as plt
import numpy as np
from numpy import arctan,tan, sin, cos
from sympy import symbols, Eq
import sympy as sp
import pandas as pd
from sympy import Eq,symbols
from IPython.display import display

### a) find the weights $w_1$ and $w_2$

In [9]:
delta_t,TM, MTcm, w_1, w_2, d_1, d_2 = symbols("\delta{t}, TM, MTcm, w_1, w_2, d_1, d_2,")

In [13]:
eq_delta_t_1 = Eq(delta_t, TM/(MTcm*100))
eq_delta_t_1

Eq(\delta{t}, TM/(100*MTcm))

In [17]:
eq_TM = Eq(TM,w_1*d_1+w_2*d_2)
eq_TM

Eq(TM, d_1*w_1 + d_2*w_2)

where $w_1$, $w_2$ are the weights $[t]$ and $d_1$, $d_2$ are the distances $[m]$ from longitudinal centre of flotation $LCF$. 

In [15]:
t_target, t_initial = symbols("t_target, t_initial")
eq_delta_t_2 = Eq(delta_t, t_target - t_initial)
eq_delta_t_2

Eq(\delta{t}, -t_initial + t_target)

In [16]:
T_AP,T_FP = symbols("T_AP,T_FP")
eq_t_initial = Eq(t_initial, T_AP - T_FP)
eq_t_initial

Eq(t_initial, T_AP - T_FP)

In [30]:
w=symbols("w")
eq_w_1 = Eq(w,w_1+w_2)
eq_w_1

Eq(w, w_1 + w_2)

In [68]:
eqs = [
    eq_delta_t_1,
    eq_TM,
    eq_delta_t_2,
    eq_t_initial,
    eq_w_1,
       ]

solution = sp.solve(eqs, delta_t, TM, w_1, w_2, t_initial, dict=True)[0]

In [69]:
equations = {symbol:Eq(symbol,expression) for symbol,expression in solution.items()}

In [70]:
for eq in equations.values():
    display(eq)

Eq(\delta{t}, -T_AP + T_FP + t_target)

Eq(TM, -100*MTcm*T_AP + 100*MTcm*T_FP + 100*MTcm*t_target)

Eq(w_1, (-100*MTcm*T_AP + 100*MTcm*T_FP + 100*MTcm*t_target - d_2*w)/(d_1 - d_2))

Eq(w_2, (100*MTcm*T_AP - 100*MTcm*T_FP - 100*MTcm*t_target + d_1*w)/(d_1 - d_2))

Eq(t_initial, T_AP - T_FP)

In [98]:
subs={
    delta_t:'delta_t',    
}
lambdas = {symbol:lambdify(eq.subs(subs)) for symbol,eq in equations.items()}

In [110]:
meta_data={
    'lpp':120,          #[m]
    'T_AP': 6.49,       #[m]
    'T_FP': 7.54,       #[m]
    'w': 600,           #[t]
    't_target': 0.2,    #[m]
    'LCF':57,           #[m]
    'TPcm':18.75,       #[t/cm]
    'MTcm':108,         #[tm/cm]
    'LCG_1':32,         #[m]
    'LCG_2':100,        #[m]
}
meta_data['d_1'] = meta_data['LCF'] - meta_data['LCG_1']  # Note that this is a distance aftward!
meta_data['d_2'] = meta_data['LCF'] - meta_data['LCG_2']  # Note that this is a distance aftward!   


In [100]:
meta_data

{'T_AP': 6.49,
 'T_FP': 7.54,
 'w': 600,
 't_target': 0.2,
 'LCF': 57,
 'TPcm': 18.75,
 'MTcm': 108,
 'LCG_1': 32,
 'LCG_2': 100,
 'd_1': 25,
 'd_2': -43}

In [101]:
calculation = {str(subs.get(symbol,symbol)):run(lambda_, meta_data) for symbol,lambda_ in lambdas.items()}

In [102]:
calculation

{'delta_t': 1.2499999999999998,
 'TM': 13500.0,
 'w_1': 577.9411764705883,
 'w_2': 22.058823529411764,
 't_initial': -1.0499999999999998}

### b) find the forward and aft drafts $T_{FP}$ and $T_{AP}$

In [103]:
delta_T, TPcm = symbols("\delta{T}, TPcm")
eq_delta_T = Eq(delta_T, w/(TPcm*100))
eq_delta_T

Eq(\delta{T}, w/(100*TPcm))

In [104]:
calculation['delta_T'] = run(lambdify(eq_delta_T), meta_data)
calculation['delta_T']

0.32

In [123]:
calculation['T_LCF_initial'] = meta_data["T_AP"]-meta_data['LCF']*calculation["t_initial"]/meta_data['lpp']  # Note -LCF
calculation['T_AP'] = calculation["T_LCF_initial"]+calculation["delta_T"] + meta_data['LCF']*meta_data["t_target"]/meta_data['lpp']
calculation['T_FP'] = calculation['T_AP']-meta_data["t_target"]

In [124]:
calculation

{'delta_t': 1.2499999999999998,
 'TM': 13500.0,
 'w_1': 577.9411764705883,
 'w_2': 22.058823529411764,
 't_initial': -1.0499999999999998,
 'delta_T': 0.32,
 'T_AP': 7.4037500000000005,
 'T_LCF_initial': 6.9887500000000005,
 'T_FP': 7.20375}

In [125]:
calculation

{'delta_t': 1.2499999999999998,
 'TM': 13500.0,
 'w_1': 577.9411764705883,
 'w_2': 22.058823529411764,
 't_initial': -1.0499999999999998,
 'delta_T': 0.32,
 'T_AP': 7.4037500000000005,
 'T_LCF_initial': 6.9887500000000005,
 'T_FP': 7.20375}

In [127]:
w = 200.1
d=-49.3
MTcm=112
t = (-w*d/(MTcm*100))
t

0.8807973214285713

In [128]:
4.95-t

4.069202678571429

In [129]:
6.989+0.32

7.309